In [31]:
sys.path.insert(1, '../../SalusCPP/src/SalGUI')

unitList = []

def ProcessTable(inDataDictTree):
    global unitList
    
    if inDataDictTree is not None:
        dataDictTree = inDataDictTree
        for table in dataDictTree.findall('Table'):
#             print(f"  {table.attrib['Name']}")
            ProcessTable(table)
        for var in dataDictTree.findall('Data_Dictionary'):
            tmpUnit = var.find('Units')
            if tmpUnit.text not in unitList:
                unitList.append(tmpUnit.text)
        

# **  Copyright (c) 2022 Michigan State University. All rights reserved. **
from SalFileClass import *

ddbFN = 'salus_AgMIP.ddb.xml'
ddbFile = SalFileClass()
ddbFile.OpenFile(ddbFN)
for fileTree in ddbFile.FileTree.findall('File'):
    print(f"** {fileTree.attrib['Name']} **")
    ProcessTable(fileTree)
print(len(unitList))
print(unitList)
convertF = open('UnitConvert.csv','w')
convertF.write('SALUS_Unit,AgMit_Unit\n')
for salUnit in unitList:
    convertF.write(f'{salUnit},\n')
convertF.close()

** CDB **
** EDB **
** GDB **
** RDB **
** SDB **
** WDB **
** XDB **
83
['N/A', 'd h^-1', 'ears plant^-1', 'ears MJ^-1', 'kernels ear^-1', 'g kernel^-1 d^-1', 'kernels ear^-1 MJ^-1', 'leaf equivalent', 'degree C', 'unitless', 'g g^-1', 'h', 'degree day leaf^-1', 'd [?]', 'g plant^-1 leaf equiv.^-1', 'C', 'm', 'leaf equivalent cm^-1', 'leaf leaf^-1', 'MJ plant^-1', None, 'g MJ^-1', 'g cm^-2', 'Mg Mg^-1', 'm^2 m^-2', '(deg. day) (deg. day)^-1', 'degree day', 'ppm', 'fraction', 'Fraction', '?', 'degree-days', 'mm', 'kg/ha', 'cm', 'plants m^-2', '% dry wt.', '%', 'd-1', 'Ratio', 'Unitless', 'PPM', 'g/plant', 'kg ha^-1', 'm^3', 'd', 'cm^2', 'day of the year', 'mm d^-1', 'hr', 'MJ m^-2 d^-1', 'count', 'kg ha^-1 d^-1', 'g', 'Mg m^-3', 'cm/day', 'm^3 m^-3', 'cm h^-1', 'g kg^-1', 'cmol kg^-1', '[?]', 'mg kg^-1', 'pH units', 'S m^-1', 'degree', 'd^-1', 'moles m^-2 d^-1', 'g/kg', 'km/d^', 'm/s', 'g elemental N Mg^-1 soil', 'g elemental P Mg^-1 soil', 'kg kg^-1', 'various', 'km d^-1', 'kg N ha^-1

In [56]:
# **  Copyright (c) 2022 Michigan State University. All rights reserved. **
import csv
sys.path.insert(1, '../../SalusCPP/src/SalGUI')
from SalFileClass import *


def ProcessTable(inDataDictTree):
    global unitCovertDict
    
    if inDataDictTree is not None:
        dataDictTree = inDataDictTree
        for table in dataDictTree.findall('Table'):
#             print(f"  {table.attrib['Name']}")
            ProcessTable(table)
        for var in dataDictTree.findall('Data_Dictionary'):
            tmpUnit = var.find('Units')
            agMIPUnitElem = ET.SubElement(var,"AgMIPUnit")
            if tmpUnit.text is not None:
                if tmpUnit.text in unitCovertDict:
                    agMIPUnit = unitCovertDict[tmpUnit.text]
                    agMIPUnitElem.text = agMIPUnit
                else:
                    print(f'didnt find {tmpUnit.text}')


inDdbFN = 'salus_AgMIP.ddb.xml'
outDdbFN = 'salus_AgMIP_Units.ddb.xml'

unitCovertF = open('UnitConvert.csv', "r")
unitCovertDict = {}
dict_reader = csv.DictReader(unitCovertF)
for row in dict_reader:
    unitCovertDict[row['SALUS_Unit']] = row['AgMit_Unit']

inDdbFile = SalFileClass()
inDdbFile.OpenFile(inDdbFN)
for fileTree in inDdbFile.FileTree.findall('File'):
    print(f"** {fileTree.attrib['Name']} **")
    ProcessTable(fileTree)

with open(outDdbFN, 'wb') as outDdbF:
    inDdbFile.FileTree.write(outDdbF)

** CDB **
** EDB **
** GDB **
** RDB **
** SDB **
** WDB **
** XDB **


In [15]:
# **  Copyright (c) 2022 Michigan State University. All rights reserved. **
import csv,sys
sys.path.insert(1, '../../SalusCPP/src/SalGUI')
from SalFileClass import *


def ProcessTable(inDataDictTree,inParentStr):
#     iDString = inParentStr
    if inDataDictTree is not None:
        dataDictTree = inDataDictTree
        for table in dataDictTree.findall('Table'):
#             print(f"  {table.attrib['Name']}")
            iDString = inParentStr + f" | Table: {table.attrib['Name']}"
            val = input(f"Look at {iDString}?")
            if val.lower() == 'y':
                ProcessTable(table,iDString)
        for var in dataDictTree.findall('Data_Dictionary'):
            varName = var.attrib['Field']
            tmpUnit = var.find('Units')
            tmpUnitAg = var.find('AgMIPUnit')
            tmpShort = var.find('Short')
            tmpVarAgMip = var.find('AgMIPName')
#             print(f'AgMIPName = {tmpVarAgMip}')
            if tmpVarAgMip is None:
                tmpVarAgMip = ET.SubElement(var,"AgMIPName")
                defaultVal = ''
            else:
                defaultVal = tmpVarAgMip.text
            tmpVarAgMip.text = (input(f"  {varName} -- {tmpShort.text} [{tmpVarAgMip.text}] ?") or tmpVarAgMip.text)
                
#             if tmpUnit.text is not None:
#                 if tmpUnit.text in unitCovertDict:
#                     agMIPUnit = unitCovertDict[tmpUnit.text]
#                     agMIPUnitElem.text = agMIPUnit
#                 else:
#                     print(f'didnt find {tmpUnit.text}')


inDdbFN = 'salus_AgMIP_Mapped.ddb.xml'
outDdbFN = 'salus_AgMIP_Mapped.ddb.xml'


inDdbFile = SalFileClass()
inDdbFile.OpenFile(inDdbFN)
for fileTree in inDdbFile.FileTree.findall('File'):
    val = input(f"Look at file {fileTree.attrib['Name']}?")
    if val.lower() == 'y':
        ProcessTable(fileTree,f"{fileTree.attrib['Name']}")

with open(outDdbFN, 'wb') as outDdbF:
    inDdbFile.FileTree.write(outDdbF)

Look at file CDB? n
Look at file EDB? n
Look at file GDB? n
Look at file RDB? n
Look at file SDB? n
Look at file WDB? n
Look at file XDB? y
Look at XDB | Table: Experiment? y
Look at XDB | Table: Experiment | Table: Experiments_Notes? y
  Notes -- Freeform notes about experiment [None] ? EXP_NARR
Look at XDB | Table: Experiment | Table: Mgt_InitialCond? y
Look at XDB | Table: Experiment | Table: Mgt_InitialCond | Table: Layer? y
  DLayrI -- Depth to bottom of the layer [None] ? ICBL
  INinorg -- Initial soil inorganic N [None] ? ICN_TOT
  P_ActIno -- Initial active inorganic P [None] ? 
  P_Labile -- Initial labile P [None] ? 
  P_SloIno -- Initial slow inorganic P [None] ? 
  SWInit -- Initial soil water content [None] ? ICH2O
  TotP -- Initial total soil P [None] ? 
  DOY -- Day of year of soil analysis [None] ? ICDAT
  EfInoc -- Rhizobia number, scaled [None] ? ICRZ#
  EfNFix -- Rhizobia effectiveness, scaled [None] ? ICRZE
  FlDD -- Initial drain depth [None] ? FLDRD
  KnDnFrac -- 

In [13]:
with open('junk.xml', 'wb') as outDdbF:
    inDdbFile.FileTree.write(outDdbF)